In [44]:


# -*- coding: utf-8 -*-
"""
Created on Wed Oct 29 15:40:04 2025

@author: zemsk


Bleaching Presence Detection
Target variable: Percent_Bleaching

"""

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import os


path = "./coral-reef-global-bleaching"
filename_read = os.path.join(path, "coral_whole.csv")

df = pd.read_csv(filename_read)

# Removed only locations and labels, no real data touched
# df.drop("Ocean_Name", axis=1, inplace=True)
# df.drop("Country_Name", axis=1, inplace=True)
# df.drop("Sample_ID", axis=1, inplace=True)
# df.drop("Date_Year", axis=1, inplace=True)
# df.drop("Bleaching_Level", axis=1, inplace=True)
# df.drop("Realm_Name", axis=1, inplace=True)
# # #Percent_Cover is not a best predictor and also contain 30% of its fields as null.
# # # For the sake of bigger dataset this feature is dropped
# df.drop("Percent_Cover", axis=1, inplace=True)
# # df.drop("ClimSST", inplace=True, axis=1)
# df.drop("Exposure", inplace=True, axis=1)
# # df.drop("Temperature_Maximum", inplace=True, axis=1)

# label_encoder = LabelEncoder()
# df["Exposure"] = label_encoder.fit_transform(df["Exposure"])

# These features were taken into account that data is nonlinear
# df = df[["Distance_to_Shore", "Temperature_Mean", "Turbidity", "TSA", "Depth_m", "Percent_Bleaching"]]

# These features were taken into account that data is linear
df = df[['Cyclone_Frequency', 'Depth_m', 'ClimSST', 'Distance_to_Shore', 'Turbidity', 'TSA', 'Temperature_Mean', 'Percent_Bleaching']]

df.dropna(inplace=True)
df.info()



<class 'pandas.core.frame.DataFrame'>
Int64Index: 32714 entries, 0 to 35044
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Cyclone_Frequency  32714 non-null  float64
 1   Depth_m            32714 non-null  float64
 2   ClimSST            32714 non-null  float64
 3   Distance_to_Shore  32714 non-null  float64
 4   Turbidity          32714 non-null  float64
 5   TSA                32714 non-null  float64
 6   Temperature_Mean   32714 non-null  float64
 7   Percent_Bleaching  32714 non-null  float64
dtypes: float64(8)
memory usage: 2.2 MB


In [2]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=['Percent_Bleaching'])
y = df['Percent_Bleaching']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [45]:
import cuml
import cudf
from cuml.ensemble import RandomForestRegressor as cuRF
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

# Convert data to cudf DataFrame (GPU equivalent of pandas DataFrame)
X_cudf = cudf.DataFrame.from_records(X)
y_cudf = cudf.Series(y)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_cudf, y_cudf, test_size=0.2, random_state=42)

# Create and train the CuML RandomForestRegressor (on GPU)
rf_model = cuRF(n_estimators=600, max_depth=30, random_state=42)
rf_model.fit(X_train, y_train)

# Make predictions
y_pred = rf_model.predict(X_test)

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test.to_array(), y_pred.to_array()))
print(f'RMSE: {rmse}')


ModuleNotFoundError: No module named 'cuml'

In [40]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(
    n_estimators=600,
    max_depth=30,
    criterion="friedman_mse",
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.,
    bootstrap=False,

    random_state=42,
    n_jobs=-1)

model.fit(X_train, y_train)

RandomForestRegressor(bootstrap=False, criterion='friedman_mse', max_depth=30,
                      n_estimators=600, n_jobs=-1, random_state=42)

In [42]:
from sklearn.metrics import r2_score, mean_absolute_error

y_pred = model.predict(X_test)

score = r2_score(y_test, y_pred)

print(score)
# print(model.oob_score_)
print(mean_absolute_error(y_test, y_pred))


0.6343814525108461
61.11841555540555


I have a feeling that ~0.65 (max 0.687) is the best performance for RandomForest. No matte how I would tackle its params or change feature set. Results tent to improve slightly, when include more features, despite big multicollinearity. Mb we could look for some more features from big file.
Could also try to take big file and apply PCA and see what happens

In [30]:
import xgboost as xgb
import numpy as np
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split, GridSearchCV, PredefinedSplit
from sklearn.metrics import mean_squared_error

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert data into DMatrix format (for GPU acceleration)
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Define parameter grid for GridSearchCV
params = {
    "max_depth": [12],  # Depth of trees
    "min_child_weight": [1, 3, 6],  # Minimum sum of instance weight (hessian) in a child
    "gamma": [0.3],  # Minimum loss reduction to make a further partition
    "subsample": [0.7, 0.8, 1.0],  # Fraction of samples to grow each tree
    "colsample_bytree": [0.5, 0.7, 1.0],  # Fraction of features to consider for each tree
    "reg_alpha": [0, 0.1, 1],  # L1 regularization
    "reg_lambda": [1, 10, 50],  # L2 regularization
    "learning_rate": [0.05, 0.1],  # Step size shrinkage
}

# Add GPU-specific parameters to the param grid
params_gpu = {
    'tree_method': ['hist'],  # Use GPU-based histogram method for faster training
    'device': ['cuda'],  # Specify GPU device (use CUDA)
}

# Combine the two parameter grids into one
combined_params = {**params, **params_gpu}

test_fold = np.concatenate([ -1 * np.ones(len(X_train)), 0 * np.ones(len(X_test)) ])


ps = PredefinedSplit(test_fold)

# Create a GridSearchCV object with the combined parameters
gs = GridSearchCV(
    xgb.XGBRegressor(),
    combined_params,
    cv=ps,
    scoring="neg_mean_absolute_error",
    verbose=1,
    n_jobs=-1
)

# Train the model using GridSearchCV with GPU
gs.fit(X_train, y_train)

# Print the best parameters from GridSearchCV
print("Best parameters from GridSearchCV:", gs.best_params_)

# Get the best model from GridSearchCV
best_model = gs.best_estimator_

# Make predictions using the best model
y_pred = best_model.predict(X_test)

# Calculate RMSE (Root Mean Squared Error)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f'Root Mean Squared Error: {rmse}')


Fitting 1 folds for each of 486 candidates, totalling 486 fits


IndexError: index 800 is out of bounds for axis 0 with size 800

In [14]:
evallist = [(dvalidation, 'validation'), (dtrain, 'train')]
num_round = 50 # Number of boosting rounds

bst = xgb.train(params, dtrain, num_round, evallist)
print("XGBoost model training complete on GPU.")

C:\Users\adjd119\AppData\Roaming\Python\Python310\site-packages\xgboost\training.py:199: UserWarning: [17:59:14] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "n_gpus", "silent" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBoostError: [17:59:14] C:\actions-runner\_work\xgboost\xgboost\src\objective\regression_obj.cu:119: Check failed: is_valid: base_score must be in (0,1) for the logistic loss.

In [37]:
import xgboost as xgb
 
 

params = {
    'colsample_bytree': 1.0, 
    'device': 'cuda', 
    'gamma': 0.3, 
    'learning_rate': 0.1, 
    'max_depth': 12, 
    'min_child_weight': 6, 
    'reg_alpha': 0.1, 
    'reg_lambda': 1, 
    'subsample': 0.7, 
    'tree_method': 'hist',
    'objective': 'reg:squarederror',  # For regression task
    'eval_metric': 'mae',  
}


xgb_model = xgb.XGBRegressor(**params)

xgb_model.fit(X_train, y_train)

bst = xgb.train(
    params_gpu,
    dtrain,
    num_boost_round=600,
    evals=[(dtrain, 'train')],
    verbose_eval=True
)

[0]	train-rmse:102.01544
[1]	train-rmse:80.17074
[2]	train-rmse:64.63814
[3]	train-rmse:52.26958
[4]	train-rmse:43.28605
[5]	train-rmse:35.79856
[6]	train-rmse:30.28236
[7]	train-rmse:26.11751
[8]	train-rmse:22.91186
[9]	train-rmse:20.46127
[10]	train-rmse:18.35595
[11]	train-rmse:16.29077
[12]	train-rmse:14.79854
[13]	train-rmse:13.67717
[14]	train-rmse:12.69799
[15]	train-rmse:12.09831
[16]	train-rmse:11.46635
[17]	train-rmse:10.87529
[18]	train-rmse:9.98162
[19]	train-rmse:9.54321
[20]	train-rmse:8.87961
[21]	train-rmse:8.48169
[22]	train-rmse:8.21489
[23]	train-rmse:8.04977
[24]	train-rmse:7.85420
[25]	train-rmse:7.60819
[26]	train-rmse:7.08710
[27]	train-rmse:6.90206
[28]	train-rmse:6.69284
[29]	train-rmse:6.60721
[30]	train-rmse:6.18555
[31]	train-rmse:6.09968
[32]	train-rmse:5.82414
[33]	train-rmse:5.67475
[34]	train-rmse:5.44975
[35]	train-rmse:5.33548
[36]	train-rmse:5.06366
[37]	train-rmse:4.98591
[38]	train-rmse:4.83178
[39]	train-rmse:4.73947
[40]	train-rmse:4.65545
[41]	tr

[332]	train-rmse:0.00105
[333]	train-rmse:0.00105
[334]	train-rmse:0.00105
[335]	train-rmse:0.00105
[336]	train-rmse:0.00105
[337]	train-rmse:0.00105
[338]	train-rmse:0.00105
[339]	train-rmse:0.00105
[340]	train-rmse:0.00105
[341]	train-rmse:0.00105
[342]	train-rmse:0.00105
[343]	train-rmse:0.00105
[344]	train-rmse:0.00105
[345]	train-rmse:0.00105
[346]	train-rmse:0.00105
[347]	train-rmse:0.00105
[348]	train-rmse:0.00105
[349]	train-rmse:0.00105
[350]	train-rmse:0.00105
[351]	train-rmse:0.00105
[352]	train-rmse:0.00105
[353]	train-rmse:0.00105
[354]	train-rmse:0.00105
[355]	train-rmse:0.00105
[356]	train-rmse:0.00105
[357]	train-rmse:0.00105
[358]	train-rmse:0.00105
[359]	train-rmse:0.00105
[360]	train-rmse:0.00105
[361]	train-rmse:0.00105
[362]	train-rmse:0.00105
[363]	train-rmse:0.00105
[364]	train-rmse:0.00105
[365]	train-rmse:0.00105
[366]	train-rmse:0.00105
[367]	train-rmse:0.00105
[368]	train-rmse:0.00105
[369]	train-rmse:0.00105
[370]	train-rmse:0.00105
[371]	train-rmse:0.00105


In [33]:
from sklearn.model_selection import GridSearchCV, PredefinedSplit

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

params_gpu = {
    'tree_method': ['hist'],
    'device': ['cuda'],
}

params = {
    "max_depth": [12],
    "min_child_weight": [1, 3, 6],
    "gamma": [0.3],
    "subsample": [0.7, 0.8, 1.0],
    "colsample_bytree": [0.5, 0.7, 1.0],
    "reg_alpha": [0, 0.1, 1],
    "reg_lambda": [1, 10, 50],
    "learning_rate": [0.05, 0.1],
}

combined_params = {**params, **params_gpu}

test_fold = np.concatenate([
    -1 * np.ones(len(X_train)),
     0 * np.ones(len(X_test))
])
ps = PredefinedSplit(test_fold)

gs = GridSearchCV(
    xgb.XGBRegressor(),
    combined_params,
    cv=ps,
    scoring="neg_mean_absolute_error",
    verbose=1,
    n_jobs=-1
)

X_all = np.concatenate([X_train, X_test])
y_all = np.concatenate([y_train, y_test])

gs.fit(X_all, y_all)
print(gs.best_params_)


Fitting 1 folds for each of 486 candidates, totalling 486 fits
{'colsample_bytree': 1.0, 'device': 'cuda', 'gamma': 0.3, 'learning_rate': 0.1, 'max_depth': 12, 'min_child_weight': 6, 'reg_alpha': 0.1, 'reg_lambda': 1, 'subsample': 0.7, 'tree_method': 'hist'}


In [43]:
y_pred = xgb_model.predict(X_test)

score = r2_score(y_test, y_pred)

print(score)
# print(model.oob_score_)
print(mean_absolute_error(y_test, y_pred))


0.9462682914935656
24.121514437845637


Compared to the Random Forest model, XGBoost improved predictive performance by 39%. Since I didnt observe strong nonlinear patterns during preprocessing, this gain likely comes from XGBoost ability to capture more complex feature interactions rather than nonlinear effects in individual variables. This is consistent with the correlation analysis: the Spearman matrix indicates that changes in some features influence others quite noticeably, while the mutual information matrix shows little evidence of direct nonlinear relationships. Taken together, it suggests that the dataset’s complexity comes primarily from interactions between features rather than from standalone nonlinearities.

<class 'pandas.core.frame.DataFrame'>
Index: 22561 entries, 6981 to 35042
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Cyclone_Frequency    22561 non-null  float64
 1   Depth_m              22561 non-null  float64
 2   ClimSST              22561 non-null  float64
 3   Ocean_Name           22561 non-null  object 
 4   Country_Name         22561 non-null  object 
 5   Distance_to_Shore    22561 non-null  float64
 6   Exposure             22561 non-null  object 
 7   Turbidity            22561 non-null  float64
 8   Date_Year            22561 non-null  int64  
 9   Bleaching_Level      22561 non-null  object 
 10  Temperature_Maximum  22561 non-null  float64
 11  SSTA                 22561 non-null  float64
 12  TSA                  22561 non-null  float64
 13  Percent_Bleaching    22561 non-null  float64
 14  Temperature_Mean     22561 non-null  float64
 15  Realm_Name           22561 non-null  o

In [40]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.preprocessing import OneHotEncoder

# Split first (no leakage)
X = df.drop(columns=['Percent_Bleaching'])
y = df['Percent_Bleaching']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [43]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(
    n_estimators=600,
    max_depth=30,
    criterion="friedman_mse",
    max_features=0.3,
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.,
    bootstrap=False,

    random_state=42,
    n_jobs=-1)

model.fit(X_train, y_train)

RandomForestRegressor(bootstrap=False, criterion='friedman_mse', max_depth=30,
                      max_features=0.3, n_estimators=600, n_jobs=-1,
                      random_state=42)

In [44]:
from sklearn.metrics import r2_score, mean_squared_error

y_pred = model.predict(X_test)

score = r2_score(y_test, y_pred)

print(score)
# print(model.oob_score_)
print(mean_squared_error(y_test, y_pred))


0.6916481848959288
99.43951094606886


I have a feeling that ~0.65 (max 0.687) is the best performance for RandomForest. No matte how I would tackle its params or change feature set. Results tent to improve slightly, when include more features, despite big multicollinearity. Mb we could look for some more features from big file.
Could also try to take big file and apply PCA and see what happens

In [112]:
import xgboost as xgb

xgb_model = xgb.XGBRegressor(
    n_estimators=500,
    max_depth=9,
    learning_rate=0.3,
    min_child_weight=1,
    gamma=0, #we dont need to add penalty since there`s enough samples I guess
    subsample=1,
    colsample_bytree=1,
    reg_alpha=0,
    reg_lambda=1,
    objective='reg:squarederror',
)

xgb_model.fit(X, y)

,objective,'reg:squarederror'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,1
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [113]:
y_pred = xgb_model.predict(X_test)

score = r2_score(y_test, y_pred)

print(score)
# print(model.oob_score_)
print(mean_squared_error(y_test, y_pred))


0.9772885931484977
7.3241378178635275


Compared to the Random Forest model, XGBoost improved predictive performance by 39%. Since I didnt observe strong nonlinear patterns during preprocessing, this gain likely comes from XGBoost ability to capture more complex feature interactions rather than nonlinear effects in individual variables. This is consistent with the correlation analysis: the Spearman matrix indicates that changes in some features influence others quite noticeably, while the mutual information matrix shows little evidence of direct nonlinear relationships. Taken together, it suggests that the dataset’s complexity comes primarily from interactions between features rather than from standalone nonlinearities.

In [3]:
from matplotlib.pyplot import imread

# -*- coding: utf-8 -*-
"""
Created on Saturday Nov 15 15:34 2025

@author: 100yearsahead


Bleaching Presence Detection
Target variable: Percent_Bleaching

"""

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import os


path = "./coral-reef-global-bleaching"
filename_read = os.path.join(path, "coral_whole.csv")

df = pd.read_csv(filename_read)

# Removed only locations and labels, no real data touched
# df.drop("Ocean_Name", axis=1, inplace=True)
# df.drop("Country_Name", axis=1, inplace=True)
# df.drop("Sample_ID", axis=1, inplace=True)
# df.drop("Date_Year", axis=1, inplace=True)
# df.drop("Bleaching_Level", axis=1, inplace=True)
# df.drop("Realm_Name", axis=1, inplace=True)
# # #Percent_Cover is not a best predictor and also contain 30% of its fields as null.
# # # For the sake of bigger dataset this feature is dropped
# df.drop("Percent_Cover", axis=1, inplace=True)
# # df.drop("ClimSST", inplace=True, axis=1)
# df.drop("Exposure", inplace=True, axis=1)
# # df.drop("Temperature_Maximum", inplace=True, axis=1)

# label_encoder = LabelEncoder()
# df["Exposure"] = label_encoder.fit_transform(df["Exposure"])

# These features were taken into account that data is nonlinear
# df = df[["Distance_to_Shore", "Temperature_Mean", "Turbidity", "TSA", "Depth_m", "Percent_Bleaching"]]

# These features were taken into account that data is linear
#df = df[['Cyclone_Frequency', 'Depth_m', 'ClimSST', 'Distance_to_Shore', 'Turbidity', 'TSA', 'Temperature_Mean', 'Percent_Bleaching']]

df.dropna(inplace=True)
df  = df.drop(columns=['Sample_ID'])
df.info()
print(df)


<class 'pandas.core.frame.DataFrame'>
Index: 22561 entries, 6981 to 35042
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Cyclone_Frequency    22561 non-null  float64
 1   Depth_m              22561 non-null  float64
 2   ClimSST              22561 non-null  float64
 3   Ocean_Name           22561 non-null  object 
 4   Country_Name         22561 non-null  object 
 5   Distance_to_Shore    22561 non-null  float64
 6   Exposure             22561 non-null  object 
 7   Turbidity            22561 non-null  float64
 8   Date_Year            22561 non-null  int64  
 9   Bleaching_Level      22561 non-null  object 
 10  Temperature_Maximum  22561 non-null  float64
 11  SSTA                 22561 non-null  float64
 12  TSA                  22561 non-null  float64
 13  Percent_Bleaching    22561 non-null  float64
 14  Temperature_Mean     22561 non-null  float64
 15  Realm_Name           22561 non-null  o

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.preprocessing import OneHotEncoder

# Split first (no leakage)
X = df.drop(columns=['Percent_Bleaching'])
y = df['Percent_Bleaching']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Since we have categorical variables we need to seperate the numeric and the categorical variables
cat_cols = ['Realm_Name','Ocean_Name','Country_Name','Exposure','Bleaching_Level']
num_cols = [col for col in X.columns if col not in cat_cols]



# We one_hot_encode the categorical features
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

X_train_cat = ohe.fit_transform(X_train[cat_cols])
X_test_cat  = ohe.transform(X_test[cat_cols])


# We scale the numeric features
scaler = StandardScaler()

X_train_num = scaler.fit_transform(X_train[num_cols])
X_test_num  = scaler.transform(X_test[num_cols])


# Combine the categorical and numerical features
X_train_processed = np.hstack([X_train_num, X_train_cat])
X_test_processed  = np.hstack([X_test_num, X_test_cat])






In [21]:
import xgboost as xgb

xgb_model = xgb.XGBRegressor(
    n_estimators=500,
    max_depth=9,
    learning_rate=0.3,
    min_child_weight=1,
    gamma=0, #we dont need to add penalty since there`s enough samples I guess
    subsample=1,
    colsample_bytree=1,
    reg_alpha=0,
    reg_lambda=1,
    objective='reg:squarederror',
)

xgb_model.fit(X_train_processed, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None, colsample_bytree=1,
             device=None, early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, feature_weights=None,
             gamma=0, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.3, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=9, max_leaves=None,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=500, n_jobs=None,
             num_parallel_tree=None, ...)

In [22]:
from sklearn.metrics import r2_score, mean_squared_error

y_pred = xgb_model.predict(X_test_processed)

score = r2_score(y_test, y_pred)

print(score)
# print(model.oob_score_)
print(mean_squared_error(y_test, y_pred))


0.8060004661717783
22.81571698634007


In [ ]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(
    n_estimators=600,
    max_depth=30,
    criterion="friedman_mse",
    max_features=0.5,
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.,
    bootstrap=False,
    random_state=42,
    n_jobs=-1)

model.fit(X_train_processed, y_train)

In [32]:
from sklearn.metrics import r2_score, mean_squared_error

y_pred = model.predict(X_test_processed)

score = r2_score(y_test, y_pred)

print(score)
# print(model.oob_score_)
print(mean_squared_error(y_test, y_pred))


0.8422120119061569
18.55698314914926


The good performance of the Random Forest model on high-dimensional feature sets can be attributed to its ensemble nature and its ability to aggregate a large number of weak but diverse predictors.

In contrast, XGBoost rely on sequential learning, where each new tree attempts to correct the residual errors of the previous ones. When the dataset contains only a few strong predictors, XGBoost can explore these features more deeply and model complex interactions, often achieving superior predictive performance.

Thus, Random Forests may outperform boosting methods in scenarios with large numbers of weak predictors, whereas XGBoost is more effective when the dataset contains fewer but more influential features.